# <center>OpenAI agent pattern: prompt chaining agent</center>

A starter guide for building an agent which chains two prompts together to generate an output using the `openai-agents` library.

In the following example, we'll build a stock portfolio creation system using this pattern:
1.  **Search Agent (Generation):** Searches the web for information on particular stock tickers.
2.  **Report Agent (Generation):** Creates a portfolio of stocks and ETFs that supports the user's investment strategy.

### Install Libraries

In [1]:
# Install base libraries for OpenAI
!pip install -q openai openai-agents pydantic

# Install optional libraries for OpenInference/OpenTelemetry tracing
!pip install -q arize-phoenix-otel openinference-instrumentation-openai-agents openinference-instrumentation-openai

### Setup Keys

Add your OpenAI API key to the environment variable `OPENAI_API_KEY`.

Copy your Phoenix `API_KEY` from your settings page at [app.phoenix.arize.com](https://app.phoenix.arize.com).

In [1]:
import os
from getpass import getpass

os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

OPENAI_API_KEY = globals().get("OPENAI_API_KEY") or getpass(
    "🔑 Enter your OpenAI API key: "
)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### Setup Tracing

In [2]:
from phoenix.otel import register

tracer_provider = register(
    project_name="openai-agents",
    endpoint="https://app.phoenix.arize.com/v1/traces",
    auto_instrument=True,
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: openai-agents
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Creating the agent



In [3]:
from textwrap import dedent
from agents import Agent, WebSearchTool, Runner, TResponseInputItem
from pydantic import BaseModel, Field
from agents.model_settings import ModelSettings
from pprint import pprint

class PortfolioItem(BaseModel):
    ticker: str = Field(
        description="The ticker of the stock or ETF."
    )
    allocation: float = Field(
        description="The percentage allocation of the ticker in the portfolio. The sum of all allocations should be 100."
    )
    reason: str = Field(
        description="The reason why this ticker is included in the portfolio."
    )

class Portfolio(BaseModel):
    tickers: list[PortfolioItem] = Field(
        description="A list of tickers that could support the user's stated investment strategy."
    )

portfolio_agent = Agent(
    name="Portfolio Agent",
    instructions=dedent("""You are a senior financial analyst. You will be provided with a stock research report. Your task is to create a portfolio of stocks and ETFs that could support the user's stated investment strategy. Include facts and data from the research report in the stated reasons for the portfolio allocation."""),
    model="o4-mini",
    output_type=Portfolio,
)

research_agent = Agent(
    name="FinancialSearchAgent",
    instructions=dedent("""You are a research assistant specializing in financial topics. Given an investment strategy, use web search to retrieve up‑to‑date context and produce a short summary of stocks that support the investment strategy at most 50 words. Focus on key numbers, events, or quotes that will be useful to a financial analyst."""),
    model="gpt-4.1",
    tools=[WebSearchTool()],
    model_settings=ModelSettings(tool_choice="required", parallel_tool_calls=True),
)

user_input = input("Enter your investment strategy: ")
input_items: list[TResponseInputItem] = [{"content": f"My investment strategy: {user_input}", "role": "user"}]


research_output = await Runner.run(research_agent, input_items)
pprint(research_output.final_output)

input_items = research_output.to_input_list()
portfolio_output = await Runner.run(portfolio_agent, input_items)
pprint(portfolio_output.final_output)

('Investing in smaller tech stocks can offer significant growth potential. '
 'Here are some small-cap tech companies to consider:\n'
 '\n'
 '**Airship AI (NASDAQ: AISP)**  \n'
 'Specializes in analyzing unstructured data, notably from surveillance '
 'videos. In 2024, the company secured contracts with the U.S. Department of '
 'Homeland Security and the Department of Justice. Revenue increased by 148% '
 'over the last 12 months, with a $130 million pipeline—five times its recent '
 'annual revenue. Analysts project a potential 185% stock price increase. '
 '([marketbeat.com](https://www.marketbeat.com/stock-ideas/ais-next-big-winners-3-small-cap-stocks-to-watch-in-2025/?utm_source=openai))\n'
 '\n'
 '**BigBear.ai (NYSE: BBAI)**  \n'
 'Provides AI-driven analytics for national security and supply chain '
 'management. In October 2024, it secured a five-year, $165 million contract '
 'with the U.S. Army. The company is also expanding into civilian sectors, '
 'implementing biometric v